In [1]:
# Import dependencies
from citipy import citipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import timeit

from datetime import datetime
from config import api_key

In [2]:
# Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(low=-80,high=80,size=2000)
lngs = np.random.uniform(low=-180,high=180,size=2000)

# Zip lats and lngs together
lat_lng = zip(lats,lngs)
lat_lng

In [3]:
# Make a list of lat_lng to find coordinates
coordinates = list(lat_lng)
print(coordinates)

[(-76.68111715673535, -56.37738635292297), (71.12123750399229, -143.73139664644327), (-6.435843183557239, -43.14836881878804), (-39.076965936201816, 80.08725466641101), (-51.401377651592156, -16.807543382241192), (-21.765930881558845, 20.149557189690682), (-7.812974794614661, 155.7827723330435), (54.228611714154454, 159.2121844891542), (-34.5559783746204, 166.7484460198873), (30.59508009978984, -179.65004631822836), (-0.45277141966751344, -112.45252737764747), (31.740442000176216, -62.39954956221122), (19.251308426417765, 147.05821819800929), (75.70459009223086, -66.62217077710591), (32.47978798267579, -119.50133967853108), (2.4721240571146836, -136.16527089858792), (-69.01083822137497, -38.75825347399399), (22.963455892379613, -122.94927321626328), (-58.44132745143334, -51.15214713214661), (70.49932645664035, -151.9681747662631), (-48.13819681800743, 33.48429802546784), (66.1561017283297, 32.76352245329252), (-32.56272545181767, -15.871316852730786), (32.02585077429845, 92.95895113747

In [4]:
# Find nearest city in citipy module
cities = []
# Use coordinate list to find cities
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    if city not in cities:
        cities.append(city)
# Print city count
len(cities)

794

In [5]:
# Perform an API call to OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

In [6]:
# Objectives for the API call:
# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | fairbanks
Processing Record 3 of Set 1 | floriano
Processing Record 4 of Set 1 | bambous virieux
Processing Record 5 of Set 1 | jamestown
Processing Record 6 of Set 1 | gobabis
Processing Record 7 of Set 1 | buin
Processing Record 8 of Set 1 | milkovo
Processing Record 9 of Set 1 | ahipara
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | lazaro cardenas
Processing Record 12 of Set 1 | saint george
Processing Record 13 of Set 1 | airai
Processing Record 14 of Set 1 | narsaq
Processing Record 15 of Set 1 | malibu
Processing Record 16 of Set 1 | atuona
Processing Record 17 of Set 1 | guerrero negro
Processing Record 18 of Set 1 | barrow
Processing Record 19 of Set 1 | port alfred
Processing Record 20 of Set 1 | loukhi
Processing Record 21 of Set 1 | lasa
Processing Record 22 of Set 1 | bethel
Processing Record 23 of Set 1 | tumannyy
City no

In [7]:
# Count lenght of cities in city_data
len(city_data)

723

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,46.06,93,100,14.97,moderate rain
1,Fairbanks,US,64.8378,-147.7164,41.00,26,0,5.75,clear sky
2,Floriano,BR,-6.7669,-43.0225,77.40,84,98,3.94,overcast clouds
3,Bambous Virieux,MU,-20.3428,57.7575,75.22,87,30,11.05,light rain
4,Jamestown,US,42.0970,-79.2353,37.53,55,0,6.91,clear sky
5,Gobabis,NA,-22.4500,18.9667,64.33,68,63,7.02,broken clouds
6,Buin,CL,-33.7333,-70.7500,71.87,60,5,2.64,clear sky
7,Milkovo,RU,54.7167,158.6167,35.19,57,50,4.61,scattered clouds
8,Ahipara,NZ,-35.1667,173.1667,69.58,91,95,13.04,light rain
9,Kapaa,US,22.0752,-159.3190,82.38,83,75,19.57,broken clouds


In [9]:
# Export the DataFrame as a CSV file
# Create the output file (CSV)
output_data_file = "../Weather_Database/WeatherPy_Database.csv"

# Export as CSV
city_data_df.to_csv(output_data_file, index_label='City ID')